In [12]:
# pc 2/20/2026
import geopandas as gpd
import pandas as pd
import os
from simpledbf import Dbf5

# =====================================================
# SETTINGS
# =====================================================

DOM_THRESHOLD = 0.85          # TAZ dominance threshold
BG_SLIVER_THRESHOLD = 0.05    # Remove BG fragments <1%

folder = r"Z:/Peter/GIS/ShapeFiles/"
bg_shp_path = os.path.join(folder, "Census/tl_2024_06_bg/tl_2024_06_bg.shp")
taz_shp_path = os.path.join(folder, "VTATAZ.shp")
acs_dbf_path = "acs2024_bg_with_gq_allocated_scaled.dbf"

output_path = "bg_taz_ratios_clean.csv"

# =====================================================
# 1. LOAD DATA
# =====================================================

print("Loading BG geometry...")
bg = gpd.read_file(bg_shp_path)

print("Loading TAZ geometry...")
taz = gpd.read_file(taz_shp_path)

print("Loading ACS DBF...")
acs_dbf = Dbf5(acs_dbf_path)
acs_df = acs_dbf.to_dataframe()

# Ensure CRS matches
if bg.crs != taz.crs:
    taz = taz.to_crs(bg.crs)

# =====================================================
# 2. INTERSECTION
# =====================================================

print("Computing intersection...")
inter = gpd.overlay(
    bg[["GEOID", "geometry"]],
    taz[["TAZ", "geometry"]],
    how="intersection"
)

# =====================================================
# 3. AREA CALCULATIONS
# =====================================================

inter["int_area"] = inter.geometry.area

bg["bg_area"] = bg.geometry.area
taz["taz_area"] = taz.geometry.area

inter = inter.merge(bg[["GEOID", "bg_area"]], on="GEOID")
inter = inter.merge(taz[["TAZ", "taz_area"]], on="TAZ")

inter["bg_ratio"] = inter["int_area"] / inter["bg_area"]
inter["taz_ratio"] = inter["int_area"] / inter["taz_area"]

# =====================================================
# 4. REMOVE BG SLIVERS
# =====================================================

inter.loc[inter["bg_ratio"] < BG_SLIVER_THRESHOLD, "bg_ratio"] = 0

# =====================================================
# 5. INITIAL ALLOCATION = bg_ratio
# =====================================================

inter["alloc_ratio"] = inter["bg_ratio"]

# =====================================================
# 6. APPLY 0.85 DOMINANCE (TAZ SIDE)
# =====================================================

# If taz_ratio >= 0.85 → that pair dominates the TAZ
#dominant_mask = inter["taz_ratio"] >= DOM_THRESHOLD

# For dominant rows:
#inter.loc[dominant_mask, "alloc_ratio"] = 1.0

# For other rows of same TAZ → set to 0
#for taz_id in inter.loc[dominant_mask, "TAZ"].unique():
#    mask = (inter["TAZ"] == taz_id)
#    inter.loc[mask & (~dominant_mask), "alloc_ratio"] = 0.0
    
    
    
# Identify dominant TAZ allocations
dominant_mask = inter["taz_ratio"] >= DOM_THRESHOLD

# Only zero out OTHER BGs for that TAZ
for taz_id in inter.loc[dominant_mask, "TAZ"].unique():
    mask = (inter["TAZ"] == taz_id)
    inter.loc[mask & (~dominant_mask), "alloc_ratio"] = 0.0

# DO NOT set dominant alloc_ratio to 1
# Leave alloc_ratio = bg_ratio
    
     

# =====================================================
# 7. REMOVE SMALL ALLOC AFTER DOMINANCE
# =====================================================

inter.loc[inter["alloc_ratio"] < BG_SLIVER_THRESHOLD, "alloc_ratio"] = 0


# =====================================================
# 7.5 BG CONTAINMENT OVERRIDE (NEW RULE)
# =====================================================

BG_CONTAIN_THRESHOLD = 0.85

# Identify BG rows that are almost fully contained in a TAZ
contain_mask = inter["bg_ratio"] >= BG_CONTAIN_THRESHOLD

# For each such BG, force full allocation to that TAZ
for geoid in inter.loc[contain_mask, "GEOID"].unique():
    
    # Identify the TAZ where containment occurs
    rows = inter[(inter["GEOID"] == geoid) & (inter["bg_ratio"] >= BG_CONTAIN_THRESHOLD)]
    
    if len(rows) > 0:
        # Choose the TAZ with largest bg_ratio (safe if multiple)
        idx = rows["bg_ratio"].idxmax()
        chosen_taz = inter.loc[idx, "TAZ"]
        
        # Zero out all allocations for this BG
        inter.loc[inter["GEOID"] == geoid, "alloc_ratio"] = 0.0
        
        # Assign full allocation to chosen TAZ
        inter.loc[
            (inter["GEOID"] == geoid) & (inter["TAZ"] == chosen_taz),
            "alloc_ratio"
        ] = 1.0


# =====================================================
# 7.6 REMOVE ISOLATED SMALL OVERLAPS (NEW REFINEMENT)
# =====================================================

BG_MINOR_THRESHOLD = 0.20
TAZ_MINOR_THRESHOLD = 0.01

small_overlap_mask = (
    (inter["bg_ratio"] < BG_MINOR_THRESHOLD) &
    (inter["taz_ratio"] < TAZ_MINOR_THRESHOLD)
)

inter.loc[small_overlap_mask, "alloc_ratio"] = 0.0
        
        
# =====================================================
# 7.7 FORCE DOMINANT BG IF CLEAR MAJORITY
# =====================================================

RATIO_THRESHOLD = 3.0   # dominant must be 3x second

for geoid, group in inter.groupby("GEOID"):

    positive = group[group["alloc_ratio"] > 0]

    if len(positive) <= 1:
        continue

    sorted_vals = positive.sort_values("alloc_ratio", ascending=False)

    max_idx = sorted_vals.index[0]
    max_val = sorted_vals.iloc[0]["alloc_ratio"]
    second_val = sorted_vals.iloc[1]["alloc_ratio"]

    if max_val / second_val >= RATIO_THRESHOLD:
        inter.loc[group.index, "alloc_ratio"] = 0.0
        inter.loc[max_idx, "alloc_ratio"] = 1.0

# =====================================================
# 8. FINAL BG NORMALIZATION (CORRECTED FOR YOUR SCRIPT)
# =====================================================

print("Applying BG normalization...")

# Ensure numeric safety
inter["alloc_ratio"] = pd.to_numeric(inter["alloc_ratio"], errors="coerce").fillna(0)

# Compute sum of positive allocations per BG
bg_sums = (
    inter[inter["alloc_ratio"] > 0]
    .groupby("GEOID")["alloc_ratio"]
    .sum()
)

# Map sums back to dataframe
inter["bg_sum"] = inter["GEOID"].map(bg_sums)

# Initialize norm_ratio
inter["norm_ratio"] = 0.0

# Normalize ONLY where alloc_ratio > 0
mask = (inter["alloc_ratio"] > 0) & (inter["bg_sum"] > 0)

inter.loc[mask, "norm_ratio"] = (
    inter.loc[mask, "alloc_ratio"] /
    inter.loc[mask, "bg_sum"]
)

# Clean temporary column
inter.drop(columns=["bg_sum"], inplace=True)


# =====================================================
# 8.5 MANUAL BG → TAZ OVERRIDE (OPTIONAL)
# =====================================================

override_file = "override_bg_taz.csv"

if os.path.exists(override_file):

    print("Applying manual BG → TAZ overrides...")

    override_df = pd.read_csv(override_file, dtype=str)

    # --- Clean bad headers (hidden spaces, tabs, etc.)
    override_df.columns = override_df.columns.str.strip()

    for _, row in override_df.iterrows():

        geoid = str(row["GEOID"]).strip()
        taz = int(str(row["TAZ"]).strip())

        # Find all rows for that BG
        mask_bg = inter["GEOID"] == geoid

        if mask_bg.any():

            # Set all TAZ shares for that BG to 0
            inter.loc[mask_bg, "norm_ratio"] = 0.0

            # Force selected TAZ to 1
            inter.loc[
                (inter["GEOID"] == geoid) &
                (inter["TAZ"] == taz),
                "norm_ratio"
            ] = 1.0

            print(f"Override applied: BG {geoid} → TAZ {taz}")

        else:
            print(f"WARNING: BG {geoid} not found in intersection table.")

else:
    print("No override_bg_taz.csv found. Skipping manual override step.")



# =====================================================
# 9. SAVE OUTPUT
# =====================================================

result = inter[["GEOID", "TAZ", "bg_ratio", "taz_ratio", "norm_ratio"]].copy()
result = result.sort_values(["GEOID", "TAZ"])

result.to_csv(output_path, index=False)

print("Saved to:", output_path)

# =====================================================
# DEBUG CHECK
# =====================================================

test_bg = "60855065023"
check = result[result["GEOID"] == test_bg]

if len(check) > 0:
    print("\nDEBUG BG:", test_bg)
    print(check)
    print("Sum of norm_ratio:", check["norm_ratio"].sum())


Loading BG geometry...
Loading TAZ geometry...
Loading ACS DBF...
Computing intersection...


<ipython-input-12-2da748635ea3>:53: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  inter["int_area"] = inter.geometry.area
<ipython-input-12-2da748635ea3>:55: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  bg["bg_area"] = bg.geometry.area
<ipython-input-12-2da748635ea3>:56: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  taz["taz_area"] = taz.geometry.area


Applying BG normalization...
Applying manual BG → TAZ overrides...
Override applied: BG 060952518041 → TAZ 2571
Override applied: BG 060952508013 → TAZ 2570
Saved to: bg_taz_ratios_clean.csv
